# Optimising the GAN to produce flow duration and flow size using Optuna

Following three examples:

1. https://towardsdatascience.com/hyperparameter-tuning-of-neural-networks-with-optuna-and-pytorch-22e179efc837
This example is a simple example to follow.

2. https://github.com/optuna/optuna-examples/blob/main/multi_objective/pytorch_simple.py
This example is more complicated but demonstrates how to have multi-objective optimisation, which is key in optimising the GAN, where we need to minimise loss for both neural networks.

3. https://gitlab.com/hpo-uq/applications/gan4hep/-/blob/main/gan4hep/train_gan_2angles.py?ref_type=heads
This example comes from the HYPPO paper, 'gan4hep' module.

In [48]:
import time
import datetime
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import pandas as pd
import optuna
print("Done")

Done


## Discriminator Class

In [49]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # Input is 2D, first hidden layer is composed of 256 neurons with ReLU activation
            nn.Linear(2, 256), 
            nn.ReLU(),

            # Have to use dropout to avoid overfitting
            nn.Dropout(0.3),

            # second and third layers are composed to 128 and 64 neurons, respectively
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # output is composed of a single neuron with sigmoidal activation to represent a probability
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        return output

## Generator Class

In [50]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        output = self.model(x)
        return output

## Load data

In [51]:
TRAINING_DATA_LENGTH = 1024

def train_data_length(data, length):
    return data[:length]
    
def load_data():
    data = torch.load("data.pt")
    data = data.to(torch.float32)
    train_data = train_data_length(data,TRAINING_DATA_LENGTH)
    return train_data
    
print("Done")  

Done


In [52]:
def train_and_optimise(params, generator, discriminator, trial):
    RANDOM_SEED = 77
    TRAINING_DATA_LENGTH = 1024

    # Device agnostic code
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # loading and moving the training data and models to the GPU if it is available
    train_data = load_data()
    train_data = train_data.to(device)
    train_labels = torch.zeros(size=(TRAINING_DATA_LENGTH, 1))
    train_labels = train_labels.to(device)
    train_set = [(train_data[i], train_labels[i]) for i in range(TRAINING_DATA_LENGTH)]
    train_loader = DataLoader(train_set, batch_size=params["batch_size"], shuffle=True, drop_last = True)

    loss_function = nn.BCELoss()
    discriminator_optimiser = optim.Adam(discriminator.parameters(), lr = params["learning_rate"])
    generator_optimiser = optim.Adam(generator.parameters(), lr = params["learning_rate"])

    if use_cuda:
        discriminator = discriminator.cuda()
        generator = generator.cuda()
        loss_function = loss_function.cuda()

    start_time = time.time()
    for epoch in range(params["epochs"]):
        
        for n, (real_samples, _) in enumerate(tqdm(train_loader)):
            # DATA FOR DISCRIMINATOR
            torch.manual_seed(RANDOM_SEED)
            real_samples_labels = torch.ones((params["batch_size"], 1), device = device)
            
            latent_space_samples = torch.randn((params["batch_size"], 2), device = device)
            generated_samples = generator(latent_space_samples)
            generated_samples_labels = torch.zeros((params["batch_size"], 1), device = device)
            
            all_samples = torch.cat((real_samples, generated_samples))
            all_samples_labels = torch.cat((real_samples_labels, generated_samples_labels))


            # TRAINING DISCRIMINATOR
            discriminator.zero_grad()
            output_discriminator = discriminator(all_samples)
            discriminator_loss = loss_function(output_discriminator, all_samples_labels)
            discriminator_loss.backward()
            discriminator_optimiser.step()


            # DATA FOR GENERATOR
            torch.manual_seed(RANDOM_SEED)
            latent_space_samples = torch.randn((params["batch_size"], 2), device = device)

            # TRAINING GENERATOR
            generator.zero_grad()
            generated_samples = generator(latent_space_samples)
            output_discriminator_generated = discriminator(generated_samples)
            generator_loss = loss_function(output_discriminator_generated, real_samples_labels)
            generator_loss.backward()
            generator_optimiser.step()

            if epoch % 10 == 0 and n == params["batch_size"]:
                print(f"Epoch: {epoch} | G. Loss: {generator_loss} | D. Loss: {discriminator_loss}")

        # Pruning
        # Need to create Pruner class for this particular case
        # trial.report(generator_loss, epoch)
        
        # if trial.should_prune():
        #    raise optuna.exceptions.TrialPruned()

    
    end_time = time.time()
    run_time = round(end_time - start_time, 2)
    print(f"Trial Complete!\nRun time for this trial was {run_time} seconds.\n")

    return generator_loss, discriminator_loss


## Objective function
The aim of this function is to define a set of hyperparameter values, build the model, train the model, and evaluate the loss of both the generator and discriminator. 

In [53]:
def objective(trial):

    params = {
        "learning_rate": trial.suggest_float("learning_rate", low=1e-5, high=1e-1, log = True),
        "batch_size": trial.suggest_int("batch_size", low=8, high=64, step=8),
        "epochs": trial.suggest_int("epochs", low=10, high=100, step=10)
    }

    discriminator = Discriminator()
    generator = Generator()

    g_loss, d_loss = train_and_optimise(params, generator, discriminator, trial)

    return g_loss, d_loss

## Display all trials and best trial

In [54]:
def display_all_trials(study):
    df = study.trials_dataframe()
    df = pd.DataFrame(df, columns = ['number', 'values_0', 'values_1', 'params_batch_size', 'params_epochs',
                                          'params_learning_rate', 'state'])
    df= df.rename(columns = {"number":"Trial #", "values_0":"G Loss", "values_1":"D Loss", "params_batch_size":"Batch Size", 
                     "params_epochs":"Epochs", "params_learning_rate":"Learning Rate", "state":"State"})
    df["Trial #"] += 1 # Adjust the trial numbers
    print(df)
    print("\n")

def display_best_trial(study):
    best_trial = study.best_trials
    best_trial_number = best_trial[0].number
    best_trial_params = best_trial[0].params
    best_trial_values = best_trial[0].values
    print("~Best trial~")
    print(f"Trial #: {best_trial_number}")
    print(f"G Loss: {best_trial_values[0]}\nD Loss: {best_trial_values[1]}")
    print(f"Batch Size: {best_trial_params['batch_size']}\nEpochs: {best_trial_params['epochs']}\nLearning Rate: {best_trial_params['learning_rate']}")

def save_trials(study):
    df = study.trials_dataframe()
    now = str(datetime.now())
    date, time = now.split(" ")
    time = time.replace(":",".")
    df.to_csv(f"Optimisation Trials/gan_optimisation_{date}_{time}.csv", index = False)
    print(f"Trials saved to: gan_optimisation_{date}_{time}.csv\n") 
    

In [55]:
def convert_time(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d hours %02d mins %02d secs" % (hour, minutes, seconds)

## Main program
The study that is created provides a multi-objective optimisation, so that it can optimise more than one value - generator loss and discriminator loss. 

In [56]:
if __name__ == "__main__":

    # Create the optimisation study
    study = optuna.create_study(directions=["minimize", "minimize"], study_name = "GAN-Optimiser", pruner=optuna.pruners.MedianPruner())

    # Optimise the objective function
    start_time = time.time()
    study.optimize(objective, n_trials = 1) # Number of trials to test with different values
    end_time = time.time()

    optimisation_run_time = convert_time(round(end_time - start_time, 2))
    
    print(f"Optimsation complete!\nOptimisation run time was {optimisation_run_time}\n")
    print("Number of finished trials: ", len(study.trials))

    save_trials(study)
    display_all_trials(study)
    display_best_trial(study)

[I 2024-01-04 18:01:24,992] A new study created in memory with name: GAN-Optimiser


  0%|          | 0/42 [00:00<?, ?it/s]

Epoch: 0 | G. Loss: 3.39906644821167 | D. Loss: 50.029754638671875


  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

[I 2024-01-04 18:01:46,064] Trial 0 finished with values: [20.944059371948242, 50.0] and parameters: {'learning_rate': 0.0005944115431248001, 'batch_size': 24, 'epochs': 50}. 


Trial Complete!
Run time for this trial was 20.29 seconds.

Optimsation complete!
Optimisation run time was 0 hours 00 mins 21 secs

Number of finished trials:  1
Trials saved to: gan_optimisation_2024-01-04_18.01.46.092247.csv

   Trial #     G Loss  D Loss  Batch Size  Epochs  Learning Rate     State
0        1  20.944059    50.0          24      50       0.000594  COMPLETE


~Best trial~
Trial #: 0
G Loss: 20.944059371948242
D Loss: 50.0
Batch Size: 24
Epochs: 50
Learning Rate: 0.0005944115431248001
